##### Copyright 2018 Google LLC.

Licensed under the Apache License, Version 2.0 (the "License");

In [1]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Install / Import / Load

This code depends on [Lucid](https://github.com/tensorflow/lucid) (our visualization library), and [svelte](https://svelte.technology/) (a web framework). The following cell will install both of them, and dependancies such as TensorFlow. And then import them as appropriate.

In [2]:
import numpy as np
import tensorflow as tf
from skimage.transform import resize
from skimage.io import imsave
from os import getcwd, listdir
from PIL import Image

import lucid.modelzoo.vision_models as models
from lucid.misc.io import show
import lucid.optvis.render as render
from lucid.misc.io import show, load
from lucid.misc.io.reading import read
from lucid.misc.io.showing import _image_url
from lucid.misc.gradient_override import gradient_override_map
import lucid.scratch.web.svelte as lucid_svelte

import sys
sys.path.append('../')
from top5gen.examples.inception_pretrained import get_top_five

Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`


Instructions for updating:
`normal` is a deprecated alias for `truncated_normal`


Load conv1_7x7_s2 weights!
Load conv1_7x7_s2 biases!
Load conv2_3x3_reduce weights!
Load conv2_3x3_reduce biases!
Load conv2_3x3 weights!
Load conv2_3x3 biases!
Load inception_3a_1x1 weights!
Load inception_3a_1x1 biases!
Load inception_3a_3x3_reduce weights!
Load inception_3a_3x3_reduce biases!
Load inception_3a_3x3 weights!
Load inception_3a_3x3 biases!
Load inception_3a_5x5_reduce weights!
Load inception_3a_5x5_reduce biases!
Load inception_3a_5x5 weights!
Load inception_3a_5x5 biases!
Load inception_3a_pool_proj weights!
Load inception_3a_pool_proj biases!
Load inception_3b_1x1 weights!
Load inception_3b_1x1 biases!
Load inception_3b_3x3_reduce weights!
Load inception_3b_3x3_reduce biases!
Load inception_3b_3x3 weights!
Load inception_3b_3x3 biases!
Load inception_3b_5x5_reduce weights!
Load inception_3b_5x5_reduce biases!
Load inception_3b_5x5 weights!
Load inception_3b_5x5 biases!
Load inception_3b_pool_proj weights!
Load inception_3b_pool_proj biases!
Load inception_4a_1x1 weigh

# Attribution Code

In [3]:
# This, obviously, loads up the model
model = models.InceptionV1()
model.load_graphdef()

In [4]:
# Here we take in the 1000-dimension vector of potential image labels
labels_str = read("https://gist.githubusercontent.com/aaronpolhamus/964a4411c0906315deb9f4a3723aac57/raw/aa66dd9dbf6b56649fa3fab83659b2acbf3cbfd1/map_clsloc.txt")
labels = [line[line.find(" "):].strip().encode() for line in labels_str.decode().split("\n")]
labels = [label.decode().strip().split()[1].replace("_", " ") for label in labels]
labels = ["dummy"] + labels

In [5]:
def raw_class_spatial_attr(img, layer, label, override=None):
  """How much did spatial positions at a given layer effect a output class?"""

  # Set up a graph for doing attribution...
  with tf.Graph().as_default(), tf.Session(), gradient_override_map(override or {}):
    t_input = tf.placeholder_with_default(img, [None, None, 3])
    T = render.import_model(model, t_input, t_input)
    
    # Compute activations
    acts = T(layer).eval()
    
    # display results

    
    if label is None: return np.zeros(acts.shape[1:-1])
    
    # Compute gradient
    softmax_result = T("softmax2_pre_activation")
    score = softmax_result[0, labels.index(label)] # The "score" is the softmax result at idx [0, label idx]
    t_grad = tf.gradients([score], [T(layer)])[0]   
    grad = t_grad.eval({T(layer) : acts})
    
    # Linear approximation of effect of spatial position
    return np.sum(acts * grad, -1)[0]

In [6]:
def raw_spatial_spatial_attr(img, layer1, layer2, override=None):
  """Attribution between spatial positions in two different layers."""

  # Set up a graph for doing attribution...
  with tf.Graph().as_default(), tf.Session(), gradient_override_map(override or {}):
    t_input = tf.placeholder_with_default(img, [None, None, 3])
    T = render.import_model(model, t_input, t_input)
    
    # Compute activations
    acts1 = T(layer1).eval()
    acts2 = T(layer2).eval({T(layer1) : acts1})
    
    # Construct gradient tensor
    # Backprop from spatial position (n_x, n_y) in layer2 to layer1.
    n_x, n_y = tf.placeholder("int32", []), tf.placeholder("int32", [])
    layer2_mags = tf.sqrt(tf.reduce_sum(T(layer2)**2, -1))[0]
    score = layer2_mags[n_x, n_y]
    t_grad = tf.gradients([score], [T(layer1)])[0]
    
    # Compute attribution backwards from each positin in layer2
    attrs = []
    for i in range(acts2.shape[1]):
      attrs_ = []
      for j in range(acts2.shape[2]):
        grad = t_grad.eval({n_x : i, n_y : j, T(layer1) : acts1})
        # linear approximation of imapct
        attr = np.sum(acts1 * grad, -1)[0]
        attrs_.append(attr)
      attrs.append(attrs_)
  return np.asarray(attrs)

# Spatial Attribution Interface

In this section, we build the *interface* for interacting with the different kinds of spatial attribution data that we can compute using the above functions. Feel free to skip over this if you aren't interested in that part. The main reason we're including it is so that you can change the interface if you want to.


In [7]:
%%html_define_svelte SpatialWidget

<div class="figure" style="width: 10000px; height: 250px; contain: strict;">
  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint1 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed3a {{pct1}}</div>
  </div>

  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint2 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed3b {{pct2}}</div>
  </div>  

  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint3 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed4a {{pct3}}</div>
  </div>

  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint4 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed4b {{pct4}}</div>
  </div>  

  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint5 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed4c {{pct5}}</div>
  </div>

 <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint6 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed4d {{pct6}}</div>
  </div>

  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint6 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed4e {{pct7}}</div>
  </div>  

  <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint8 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed5a {{pct8}}</div>
  </div>

    <div class="outer" on:mouseleave="set({pos2: undefined})">
    <img class="img"  src="{{img}}">
    <img class="attr" src="{{(pos1 == undefined)? hint9 : spritemap1[pos1[1]][pos1[0]]}}">

    <div class="label">mixed5b {{pct9}}</div>
  </div>
  
</div>


<style>

  .outer{
    width: 224px;
    height: 224px;
    display: inline-block;
    margin-right: 2px;
    position: relative;
  }
  .outer img, .outer svg {
    position: absolute;
    left: 0px;
    top: 0px;
    width: 224px;
    height: 224px;
    image-rendering: pixelated; 
  }
  .attr {
    opacity: 0.6;
  }
  .pointer_container {
    z-index: 100;
  }
  .pointer_container rect {
    opacity: 0;
  }
  .pointer_container .selected  {
    opacity: 1;
    fill: none;
    stroke: hsl(24, 100%, 50%);
    stroke-width: 0.1px;
  }
  .label{
    position: absolute;
    left: 0px;
    top: 224px;
    width: 224px;
  }
</style>

<script>
  function range(n){
    return Array(n).fill().map((_, i) => i);
  }
  
  export default {
    data () {
      return {
        img: "",
        hint1: "",
        hint2: "",
        hint3: "",
        hint4: "",
        hint5: "",
        hint6: "",
        hint7: "",
        hint8: "",
        hint9: "",
        pct1: "",
        pct2: "",
        pct3: "",
        pct4: "",
        pct5: "",
        pct6: "",
        pct7: "",
        pct8: "",
        pct9: "",
        spritemap1 : "",
        size1: 1,
        spritemap2 : "",
        size2: 1,
        pos1: undefined,
        pos2: undefined,
        layer1: "",
        layer2: ""
      };
    },
    computed: {
      xs1: (size1) => range(size1),
      ys1: (size1) => range(size1),
      xs2: (size2) => range(size2),
      ys2: (size2) => range(size2)
    },
    helpers: {range}
  };
</script>

Trying to build svelte component from html...
svelte compile --format iife /tmp/svelte_o7115rg6/SpatialWidget_cbd471db_3182_44ec_a153_96e43225b606.html > /tmp/svelte_o7115rg6/SpatialWidget_cbd471db_3182_44ec_a153_96e43225b606.js
b'svelte version 1.64.1\ncompiling ../../../../tmp/svelte_o7115rg6/SpatialWidget_cbd471db_3182_44ec_a153_96e43225b606.html...\n(4:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(5:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(11:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(12:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(18:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(19:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(25:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(26:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(32:4) \xe2\x80\x93 A11y: <img> element should have an alt attribute\n(33:4) \xe2\x80\

In [8]:
def orange_blue(a,b,clip=False):
  """This is what makes the orange/blue highlights in the saliency map"""
  
  # Clip *is* called on each run in the given code
  # --This appears to cut down on noise (in purple) in the final output
  if clip:
    a,b = np.maximum(a,0), np.maximum(b,0)
    
  # This is all original to what I was given, I don't know what the constants represent
  img = np.stack([a, (a + b)/2., b], -1)
  img /= 1e-2 + np.abs(img).max()/1.5
  img += 0.3  # Brightens the image a tad
  img[img>1] = 1 # Truncate oddities
  highlight_count = 0
  pixel_count = 0
  
  # Gets the number of pixels that aren't black
  # Due to some image brightening, 'black' is considered 0.3
  for row in img:
    for pixel in row:
        pixel_count += 1
        for rgbval in pixel:
          if rgbval > 0.3:
            highlight_count += 1
            break
  
  return [img, highlight_count/pixel_count]

# Simple Attribution Example

In [9]:
def image_url_grid(grid):
  return [[_image_url(img) for img in line] for line in grid ]

In [13]:
# This block gets the saliency maps themselves

layers = ['mixed3a', 'mixed3b', 'mixed4a', 'mixed4b', 'mixed4c', 'mixed4d', 'mixed4e', 'mixed5a', 'mixed5b']

# Use for single input dir
# cur_labels = get_top_five("/home/w266ajh/Documents/top5gen/data_imageparts", ".jpg")

# Use for multiple input dirs
# Root dir
root_dir = "/home/w266ajh/Documents/top5gen/similar-images/"

# Go thorugh each dir in root dir
for dir in listdir(root_dir):
    go = False
    print(dir)
    # Continue if dir is empty
    if listdir(root_dir + dir) == list(): continue
    # Continue if no jpgs
    for file in listdir(root_dir + dir):
        if file.endswith(".jpg"): go = True
    if not go: continue
    cur_labels = get_top_five(root_dir + dir, '.jpg')
    for label in cur_labels:
        # Note: Must save the file off each time because it will not re-read raw ndarray
        file = label.pop(0)
        img = resize(load(file), (224, 224))  # Make all 224x224
        imsave("current.jpeg", img, plugin='pil', format_str='jpeg')
        img = load("current.jpeg")
        for tag in label:
            print(tag)
            saliency_layers = []
            saliency_values = []
            for layer in layers:
                # Get our saliency layer
                saliency = orange_blue(
                  raw_class_spatial_attr(img, layer, tag.split(':')[-1].strip(), override=None),
                  raw_class_spatial_attr(img, layer, None, override=None),
                  clip=True
                )
                saliency_layers.append(saliency[0])
                saliency_values.append(saliency[1])
                imsave("../saliency_maps/" + file.split('/')[-1] + '_' + layer + ".png", saliency[0], plugin='pil', format_str='png')

                attrs = raw_spatial_spatial_attr(img, layer, "mixed5b", override=None)
                attrs = attrs / attrs.max()
                break
            break

Mashedpotato


/home/w266ajh/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:105: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "
/home/w266ajh/.local/lib/python3.6/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


1: probability: 1.00, label: cauliflower
1: probability: 1.00, label: cauliflower
1: probability: 1.00, label: cauliflower
Stocking
1: probability: 1.00, label: mitten
1: probability: 0.96, label: mitten
1: probability: 0.97, label: mitten
1: probability: 0.98, label: sock
1: probability: 0.99, label: sock
1: probability: 1.00, label: sock
Schoolbus
1: probability: 0.99, label: ambulance
1: probability: 0.91, label: ambulance
1: probability: 0.97, label: ambulance
Gaspump
1: probability: 0.98, label: cash machine
1: probability: 0.97, label: cash machine
1: probability: 1.00, label: cash machine
1: probability: 0.96, label: pay-phone
1: probability: 0.97, label: pay-phone
1: probability: 0.95, label: pay-phone
Sliderule
1: probability: 0.96, label: computer keyboard
1: probability: 0.91, label: computer keyboard
1: probability: 0.96, label: computer keyboard
1: probability: 0.99, label: rule
1: probability: 1.00, label: rule
1: probability: 1.00, label: rule
Colobus
1: probability: 0.9

In [14]:
def highlight_interest_areas(saliency, input_image):
    # Load in our image
    img = np.array(Image.open(saliency).resize((224,224))) / 255  # Adjust input saliency here
    

    # Eliminate the low values--leave only high
    img[img<0.5] = 0  # Adjust the dropout here

    # Save off modified saliency
    imsave("testing.png", img, plugin='pil', format_str='png')

    ###

    # Get our overlay image
    img_pastable = Image.open(input_image)  # Adjust input image here
    img_pastable = img_pastable.resize((224, 224), Image.ANTIALIAS)
    img_overlay = Image.open("testing.png")
    img_overlay = img_overlay.resize((224, 224))
    img_overlay = img_overlay.convert("RGBA")
    datas = img_overlay.getdata()

    newData = []
    for item in datas:
        if item[0] == 0 and item[1] == 0 and item[2] == 0:
            newData.append((0, 0, 0, 0))
        else:
            newData.append(item)

    img_overlay.putdata(newData)

    img_pastable.paste(img_overlay, (0,0), img_overlay)
    img_pastable.save('overlaid', 'png', quality=100)

    ###

    # Get the overlaid region out of the original image
    x, y, z = (img != 0).nonzero()

    # Iterate through the photo, using only the things ID'd as not highlighted
    photo_chunk = np.zeros((224,224,3))
    photo = np.array(Image.open(input_image).resize((224,224), Image.ANTIALIAS)) / 255	# Adjust input image here
    imsave("testing_chunks.png", photo, plugin='pil', format_str='png')

    for idx, subx in enumerate(x):
      for i in range(0, 3):
        photo_chunk[subx, y[idx], i] = photo[subx, y[idx], i]
    imsave("/home/w266ajh/Documents/output_interests/" + input_image.split('/')[-1], photo_chunk, plugin='pil', format_str='png')

In [32]:
# Single file path
# root_img_path = "/home/w266ajh/Documents/top5gen/data_imageparts"

# Multiple file paths
# Root dir
root_img_path = "/home/w266ajh/Documents/top5gen/similar-images/"

# Go thorugh each dir in root dir
image_list = []
for dir in listdir(root_dir):
    go = False
    print(dir)
    # Continue if dir is empty
    if listdir(root_dir + dir) == list(): continue
    # Continue if no jpgs
    for file in listdir(root_dir + dir):
        if file.endswith(".jpg"): go = True
    if not go: continue
    image_list += [root_img_path + dir + '/' + file for file in listdir(root_dir + dir)
                    if file.endswith(".jpg")]
root_sal_path = "/home/w266ajh/Documents/saliency_maps"
# image_list = sorted([image for image in listdir(root_img_path) if image.endswith(".jpg")])
saliency_list = sorted([sal for sal in listdir(root_sal_path)])
for idx, img in enumerate(sorted(image_list, key=lambda path: path.split('/')[-1])):
    print(img)
    print(saliency_list[idx])
    highlight_interest_areas(root_sal_path + '/' + saliency_list[idx], img)

Mashedpotato
Stocking
Schoolbus
Gaspump
Sliderule
Colobus
Pier
Watertower
Train
Polaroid
Telephone
Burger
Barn
Dam
Weevil
Nematode
/home/w266ajh/Documents/top5gen/similar-images/Schoolbus/ambulance1.jpg
ambulance1.jpg_mixed3a.png
/home/w266ajh/Documents/top5gen/similar-images/Schoolbus/ambulance2.jpg
ambulance2.jpg_mixed3a.png
/home/w266ajh/Documents/top5gen/similar-images/Schoolbus/ambulance3.jpg
ambulance3.jpg_mixed3a.png


/home/w266ajh/.local/lib/python3.6/site-packages/skimage/util/dtype.py:141: UserWarning: Possible precision loss when converting from float64 to uint8
  .format(dtypeobj_in, dtypeobj_out))


/home/w266ajh/Documents/top5gen/similar-images/Gaspump/atm1.jpg
atm1.jpg_mixed3a.png
/home/w266ajh/Documents/top5gen/similar-images/Gaspump/atm2.jpg
atm2.jpg_mixed3a.png
/home/w266ajh/Documents/top5gen/similar-images/Gaspump/atm3.jpg
atm3.jpg_mixed3a.png
/home/w266ajh/Documents/top5gen/similar-images/Watertower/balloon1.jpg
balloon1.jpg_mixed3a.png
/home/w266ajh/Documents/top5gen/similar-images/Watertower/balloon2.jpg
balloon2.jpg_mixed3a.png
/home/w266ajh/Documents/top5gen/similar-images/Watertower/balloon3.jpg
balloon3.jpg_mixed3a.png
/home/w266ajh/Documents/top5gen/similar-images/Barn/boathouse1.jpg
boathouse1.jpg_mixed3a.png
/home/w266ajh/Documents/top5gen/similar-images/Barn/boathouse2.jpg
boathouse2.jpg_mixed3a.png
/home/w266ajh/Documents/top5gen/similar-images/Barn/boathouse3.jpg
boathouse3.jpg_mixed3a.png
/home/w266ajh/Documents/top5gen/similar-images/Train/bobsled1.jpg
bobsled1.jpg_mixed3a.png
/home/w266ajh/Documents/top5gen/similar-images/Train/bobsled2.jpg
bobsled2.jpg_mixed

In [ ]:
getcwd()